In [45]:
# !pip install missingno
# !pip install geopy

In [46]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [47]:
from google.colab import drive
drive.mount('/content/drive')

In [48]:
AirbnbBerlin_df = pd.read_csv('/content/drive/My Drive/Airbnb/Airbnb Berlin.csv', index_col=0)
# AirbnbBerlin_df = pd.read_csv('./content/Airbnb Berlin.csv')
# df_2019 = AirbnbBerlin_df[AirbnbBerlin_df['review_date'].astype(str).str[6:] == '19']

# df_2019 = pd.read_csv('/content/drive/My Drive/Airbnb/AirbnbBerlin_2019.csv', index_col=0)
# df_2019 = pd.read_csv('./content/AirbnbBerlin_2019.csv')

In [49]:
df = AirbnbBerlin_df.copy()

# 1. Data Preparation

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456961 entries, 0 to 456960
Data columns (total 47 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   index                  456961 non-null  int64  
 1   Review ID              452805 non-null  float64
 2   review_date            452805 non-null  object 
 3   Reviewer ID            452805 non-null  float64
 4   Reviewer Name          452805 non-null  object 
 5   Comments               452595 non-null  object 
 6   Listing ID             456961 non-null  int64  
 7   Listing URL            456961 non-null  object 
 8   Listing Name           456756 non-null  object 
 9   Host ID                456961 non-null  int64  
 10  Host URL               456961 non-null  object 
 11  Host Name              456913 non-null  object 
 12  Host Since             456913 non-null  object 
 13  Host Response Time     398194 non-null  object 
 14  Host Response Rate     398194 non-nu

### Aggregate dataset by Listing ID

Clean text From pancutations or undesired characters

In [51]:
# 1. Clean Text: Perform text cleaning, remove currency symbols & commas
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)
df['Host Response Rate'] = df['Host Response Rate'].replace('[\%,]', '', regex=True).astype(float)

# Fix Postal Code incorrect values, remove '\n' and other irrelevant text
df['Postal Code'] = df['Postal Code'].astype(str).str[:5]

Adjust Prices for Inflation

In [52]:
# Example CPI data
cpi_data = {
    2009: 92.2,
    2010: 93.2,
    2011: 95.2,
    2012: 97.1,
    2013: 98.5,
    2014: 99.4,
    2015: 100.0,
    2016: 100.5,
    2017: 102.0,
    2018: 103.8,
    2019: 105.3
}

# Base year for adjustment
base_year = 2019
base_cpi = cpi_data[base_year]

# Function to adjust price for inflation
def adjust_for_inflation(row):
    original_year = row['Review Date Year']
    original_price = row['Price']
    if original_year in cpi_data:
        original_cpi = cpi_data[original_year]
        adjusted_price = original_price * (base_cpi / original_cpi)
        return round(adjusted_price)
    else:
        return round(original_price)  # If year not in CPI data, return original price

df['review_date'] = pd.to_datetime(df['review_date'])
df['Review Date Year'] = df['review_date'].dt.year

# Apply the adjustment
df['Adjusted Price'] = df.apply(adjust_for_inflation, axis=1)

Aggregate the dataset by 'Listing ID'
- For numerical columns, we'll compute the mean
- For categorical columns, we'll take the first value (assuming consistency)

In [53]:
# Aggregate the dataset by 'Listing ID'
# For numerical columns, we'll compute the mean
# For categorical columns, we'll take the first value (assuming consistency)
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['number']).columns.difference(['Listing ID'])
aggregated_df = df.groupby('Listing ID').agg({**{col: 'mean' for col in numerical_cols},
                                              **{col: 'first' for col in categorical_cols}})

# aggregated_df = df.groupby('Listing ID').agg({**{col: 'mean' for col in numerical_cols},
#                                               **{col: lambda x: x.mode().iloc[0] if not x.mode().empty else None for col in categorical_cols}})

print(aggregated_df.shape)

(23536, 47)


### Features Selection
1. If a categorical column is not relevant to the analysis, we can remove it.
2. Listing URL, Listing Name, Host URL, Host Name: These are mostly unique to each listing, so not useful for category reduction

In [54]:
aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23536 entries, 2695 to 34682315
Data columns (total 47 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Accomodates            23536 non-null  float64
 1   Accuracy Rating        18888 non-null  float64
 2   Adjusted Price         23536 non-null  float64
 3   Bathrooms              23507 non-null  float64
 4   Bedrooms               23516 non-null  float64
 5   Beds                   23501 non-null  float64
 6   Checkin Rating         18870 non-null  float64
 7   Cleanliness Rating     18892 non-null  float64
 8   Communication Rating   18886 non-null  float64
 9   Guests Included        23536 non-null  float64
 10  Host ID                23536 non-null  float64
 11  Host Response Rate     13046 non-null  float64
 12  Latitude               23536 non-null  float64
 13  Location Rating        18871 non-null  float64
 14  Longitude              23536 non-null  float64
 15  M

In [55]:
# drop the columns that is not helpful for prediction
df = aggregated_df.copy()
df = df.drop(columns=['Review ID', 'Reviewer ID', 'Reviewer Name', 'Listing URL','Listing Name',
                      'Host ID', 'Host URL', 'Host Name', 'City', 'Country Code', 'Country',
                      'First Review', 'Last Review', 'Square Feet', 'Business Travel Ready'])

### Reduce Large Categories

1. Group Rare Categories: If a categorical column has many unique values, we can group infrequent categories into an "Other" category like 'Reviewer Name'.
2. Merge Similar Categories: If there are similar categories (e.g., different spellings or formats of the same category), we can merge them.
3. Binning: For numerical categories (like "Overall Rating" or "Accommodates"), we can create bins to reduce the number of unique values.

In [56]:
# 2. Grouping neighbourhoods into Neighborhood Groups
if 'Neighborhood Group' in df.columns:
  neighbourhood_mapping = df.groupby('neighbourhood')['Neighborhood Group'].first()
  df['Neighbourhood Grouped'] = df['neighbourhood'].map(neighbourhood_mapping)

# 3. Reducing Property Types
property_mapping = {
    "Villa": "Vacation Rental",
    "Cottage": "Vacation Rental",
    "Bungalow": "Vacation Rental",
    "Cabin": "Vacation Rental",
    "Tiny house": "Vacation Rental",
    "Earth house": "Vacation Rental",
    "Treehouse": "Vacation Rental",
    "Hut": "Vacation Rental",
    "Barn": "Vacation Rental",
    "Houseboat": "Boats & Houseboats",
    "Boat": "Boats & Houseboats",
    "Camper/RV": "Mobile/Alternative Lodging",
    "Cave": "Mobile/Alternative Lodging",
    "Pension (South Korea)": "Mobile/Alternative Lodging",
    "Casa particular (Cuba)": "Mobile/Alternative Lodging",
}

# Apply mapping and assign 'Other' to rare categories
top_property_types = [
    "Apartment", "Loft", "House", "Townhouse", "Condominium", "Serviced apartment",
    "Hotel", "Hostel", "Guesthouse", "Bed and breakfast", "Boutique hotel"
]

df['Property Type Reduced'] = df['Property Type'].apply(
    lambda x: property_mapping.get(x, x) if x in top_property_types or x in property_mapping else "Other"
)

# 4. Binning Postal Codes (first two digits represent broad area)
df['Postal Code Reduced'] = df['Postal Code'].astype(str).str[:2]  # Use only first 2 digits

### Transform/Manipulate data

In [57]:
# Extracting years from date columns
df['Host Since'] = pd.to_datetime(df['Host Since'])
df['Host Since Year'] = df['Host Since'].dt.year

# 5. transform true/false into bool
df['Instant Bookable'] = df['Instant Bookable'].replace({'t': True, 'f': False})
df['Is Superhost'] = df['Is Superhost'].replace({'t': True, 'f': False})
df['Is Exact Location'] = df['Is Exact Location'].replace({'t': True, 'f': False})
df['Instant Bookable'] = df['Instant Bookable'].replace({'t': True, 'f': False})

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23536 entries, 2695 to 34682315
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Accomodates            23536 non-null  float64       
 1   Accuracy Rating        18888 non-null  float64       
 2   Adjusted Price         23536 non-null  float64       
 3   Bathrooms              23507 non-null  float64       
 4   Bedrooms               23516 non-null  float64       
 5   Beds                   23501 non-null  float64       
 6   Checkin Rating         18870 non-null  float64       
 7   Cleanliness Rating     18892 non-null  float64       
 8   Communication Rating   18886 non-null  float64       
 9   Guests Included        23536 non-null  float64       
 10  Host Response Rate     13046 non-null  float64       
 11  Latitude               23536 non-null  float64       
 12  Location Rating        18871 non-null  float64       
 13  

In [59]:
df.head(2)

,Accomodates,Accuracy Rating,Adjusted Price,Bathrooms,Bedrooms,Beds,Checkin Rating,Cleanliness Rating,Communication Rating,Guests Included,Host Response Rate,Latitude,Location Rating,Longitude,Min Nights,Overall Rating,Price,Review Date Year,Reviews,Value Rating,index,Comments,Host Since,Host Response Time,Is Superhost,neighbourhood,Neighborhood Group,Postal Code,Is Exact Location,Property Type,Room Type,Instant Bookable,Neighbourhood Grouped,Property Type Reduced,Postal Code Reduced,Host Since Year
Listing ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2695,2.0,10.0,17.000000,1.0,1.0,1.0,10.0,10.0,10.0,1.0,50.0,52.54851,9.0,13.40455,2.0,100.0,17.0,2018.142857,7.0,10.0,271261.0,I really enjoyed staying at Micha and Nadja's ...,2008-09-16,within a day,False,Prenzlauer Berg,Pankow,10437,True,Apartment,Private room,False,Pankow,Apartment,10,2008.0
3176,4.0,9.0,96.097222,1.0,1.0,2.0,9.0,9.0,9.0,2.0,50.0,52.53500,10.0,13.41758,62.0,92.0,90.0,2013.805556,144.0,9.0,306635.5,"excellent stay, i would highly recommend it. a...",2008-10-19,within a day,False,Prenzlauer Berg,Pankow,10405,True,Apartment,Entire home/apt,False,Pankow,Apartment,10,2008.0


### EDA (Exploratory Data Analysis)

In [62]:
df_EDA = df.drop(columns=['Host Since', 'neighbourhood', 'Property Type', 'Postal Code'])
# 'Comments','Instant Bookable', 'Is Superhost', 'Is Exact Location'
# df_EDA.to_csv('./content/df_EDA.csv')
df_EDA.to_csv('/content/drive/My Drive/Airbnb/df_EDA.csv')

Store EDA deady dataset as pkl file

In [63]:
# import pickle
# with open('./content/df_EDA.pkl', 'wb') as f:
#   pickle.dump(df_EDA, f)

In [ ]:
import pickle
with open('/content/drive/My Drive/Airbnb/df_EDA.pkl', 'wb') as f:
    pickle.dump(df_EDA, f)